<a href="https://colab.research.google.com/github/LiuChen-5749342/Foundations-of-Finance/blob/main/Coding/workshop_CAPM_and_multifactor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Python Workshop 2: Sharpe Ratio, CAPM, and Multi-factor models


In [ ]:
# Import libraries
import pandas as pd # data frames
import numpy as np # numerical computing
import matplotlib.pyplot as plt # plotting
from pathlib import Path # path management

In [ ]:
# Define project directories
#path_project = Path.cwd().parent # For local machine
path_project = Path("/content/") # for Google Colab
path_code = path_project / "code/"
path_input = path_project / "input/"
path_output = path_project / "output/"

print(f"path_project: {path_project}")
print(f"path_code: {path_code}")
print(f"path_input: {path_input}")
print(f"path_output: {path_output}")

In [ ]:
# Import CRSP monthly data subset
crsp_subset = pd.read_csv(path_input / "crsp_monthly_workshop.csv")

In [ ]:
# Print data frame
crsp_subset

### Clean data

In [ ]:
# Convert date to datetime format
crsp_subset['date'] = pd.to_datetime(crsp_subset['date'])

In [ ]:
# Clean price and return columns
crsp_subset['prc'] = crsp_subset['prc'].abs()
crsp_subset['ret'] = pd.to_numeric(crsp_subset['ret'], errors='coerce')

In [ ]:
# Compute market capitalization
crsp_subset['mktcap'] = crsp_subset['prc'] * crsp_subset['shrout'] / 1e6  # in billions

In [ ]:
# Get unique permnos
permno_subset = crsp_subset['permno'].unique()

### Open dataset with mean returns and standard deviations of selected stocks from previous workshop

In [ ]:
# Load return statistics
return_stats = pd.read_csv(path_input / "return_stats.csv")

print(return_stats)

In [ ]:
# Plot assets in risk-return space
plt.figure(figsize=(10, 6))
plt.scatter(return_stats['std_ret'], return_stats['mean_ret'], alpha=0.7)
for i in range(return_stats.shape[0]):
    plt.annotate(return_stats['asset_name'].values[i], (return_stats['std_ret'].values[i], return_stats['mean_ret'].values[i]))
plt.title("Historical mean and standard deviation of monthly returns", fontsize=16)
plt.xlabel("Standard Deviation (Risk)")
plt.ylabel("Mean Monthly Return")
plt.xlim(0, return_stats['std_ret'].max() * 1.5)
plt.ylim(0, return_stats['mean_ret'].max() * 1.5)
plt.grid()
plt.show()

### Prepare dataset with factor returns and risk-free rate
 - You can find up-to-date factor returns data on the website of Kenneth French:
 https://mba.tuck.dartmouth.edu/pages/faculty/ken.french/data_library.html
 - For this exercise, we will need data on *monthly* Fama/French 5 Factors and Momentum Factor
    1) Fama/French 5 Factors (2x3)
    2) Momentum Factor (Mom)
 - File on Fama/French 5 factors also contains a column for risk-free rate


In [ ]:
# Import FF5 returns data
# Note: careful with skiprows and nrows parameters to get the correct data
# - inspect the CSV file to determine these values
ff5_data = pd.read_csv(path_input / "F-F_Research_Data_5_Factors_2x3.csv", skiprows=3, nrows=746)

In [ ]:
# Rename first column to 'date'
ff5_data.rename(columns={ff5_data.columns[0]: 'date',
                         ff5_data.columns[1]: 'mktrf'}, inplace=True)
# Make all other columns lowercase
ff5_data.columns = [col.lower() for col in ff5_data.columns]
# Convert date to datetime format assuming that the data is valid as of the end of the month
ff5_data['date'] = pd.to_datetime(ff5_data['date'], format='%Y%m') + pd.offsets.MonthEnd(0)


In [ ]:
# Import Momentum factor returns data
# Note: careful with skiprows and nrows parameters to get the correct data
# - inspect the CSV file to determine these values
mom_data = pd.read_csv(path_input / "F-F_Momentum_Factor.csv", skiprows=13, nrows=1184)
mom_data.rename(columns={mom_data.columns[0]: 'date',
                         mom_data.columns[1]: 'mom'}, inplace=True)
# Convert date to datetime format assuming that the data is valid as of the end of the month
mom_data['date'] = pd.to_datetime(mom_data['date'], format='%Y%m') + pd.offsets.MonthEnd(0)

In [ ]:
# Merge FF5 and Momentum data
factor_data = pd.merge(ff5_data, mom_data, on='date', how='left')

In [ ]:
# Since FF5 data are in percentages, convert them to decimals
factor_data[['mktrf', 'smb', 'hml', 'rmw', 'cma', 'mom', 'rf']] = factor_data[['mktrf', 'smb', 'hml', 'rmw', 'cma','mom', 'rf']] / 100

### Merge individual stock data with factor return data

In [ ]:
# Merge on year and month
# - convert date to yearmonth variable
crsp_subset['yearmonth'] = crsp_subset['date'].dt.to_period('M')
factor_data['yearmonth'] = factor_data['date'].dt.to_period('M')

# - perform the merge
crsp_factors = pd.merge(crsp_subset, factor_data.drop(columns=['date']), on='yearmonth', how='left')

In [ ]:
# Compute excess returns for individual stocks
# - denote excess returns as 'retex'
crsp_factors['retex'] = crsp_factors['ret'] - crsp_factors['rf']

In [ ]:
# Compute average monthly return, excess return, and standard deviation of returns for individual stocks
return_stats = crsp_factors.groupby('permno').agg(
    mean_ret=('ret', 'mean'),
    mean_retex=('retex', 'mean'),
    std_ret=('ret', 'std')
)

# Add the last comnam for each permno
last_comnam = crsp_factors.groupby('permno')['comnam'].last().reset_index()
return_stats = return_stats.merge(last_comnam, on='permno')
return_stats.columns = ['permno', 'mean_ret', 'mean_retex', 'std_ret', 'asset_name']

In [ ]:
# Construct equally-weighted and value-weighted portfolios
# - ensure data is sorted by date and permno
crsp_factors = crsp_factors.sort_values(by=['date', 'permno'])
# - get lagged market capitalization
crsp_factors['mktcap_l1'] = crsp_factors.groupby('permno')['mktcap'].shift(1)
# - value weighted portfolio weights
crsp_factors['w_vw'] = crsp_factors['mktcap_l1'] / crsp_factors.groupby('date')['mktcap_l1'].transform('sum')
# - equally weighted portfolio weights
crsp_factors['w_ew'] = 1 / crsp_factors.groupby('date')['permno'].transform('count')

In [ ]:
# Compute returns for equally-weighted and value-weighted portfolios
crsp_factors['ret_ew'] = (crsp_factors['ret'] * crsp_factors['w_ew']).groupby(crsp_factors['date']).transform('sum')
crsp_factors['ret_vw'] = (crsp_factors['ret'] * crsp_factors['w_vw']).groupby(crsp_factors['date']).transform('sum')

In [ ]:
# Compute average risk-free rate over the sample period
# Note: the sample period over which the mean and SD was computed in the previous workshop is Jan 2010-Dec 2024
rf_mean_subsample = factor_data[(factor_data['date'] >= "2010-01-01") & (factor_data['date'] <= "2024-12-31")]['rf'].mean()

In [ ]:
# Append portfolio returns to return_stats
# - subset to monthly portfolio returns
portfolio_stats = crsp_factors[(crsp_factors['date'] >= "2010-01-01") & \
                               (crsp_factors['date'] <= "2024-12-31")].groupby('date')[['ret_ew', 'ret_vw']].first().reset_index()
# - compute mean and std dev
ew_mean = portfolio_stats['ret_ew'].mean()
ew_mean_ex = ew_mean - rf_mean_subsample
ew_std = portfolio_stats['ret_ew'].std()
vw_mean = portfolio_stats['ret_vw'].mean()
vw_mean_ex = vw_mean - rf_mean_subsample
vw_std = portfolio_stats['ret_vw'].std()
# - convert into DataFrame
portfolio_summary = pd.DataFrame({
    'mean_ret': [ew_mean, vw_mean],
    'mean_retex': [ew_mean_ex, vw_mean_ex],
    'std_ret': [ew_std, vw_std],
    'asset_name': ['PORTF EW', 'PORTF VW'],
})
# - concatenate along axis 0 (rows)
return_stats = pd.concat([return_stats, portfolio_summary], ignore_index=True)

In [ ]:
print(return_stats)

### Compute Sharpe ratios of individual stocks, EW and VW portfolios

In [ ]:
# Compute monthly Sharpe ratios of individual stocks, EW and VW portfolios
return_stats['sharpe_ratio_MON'] = return_stats['mean_retex'] / return_stats['std_ret']

In [ ]:
# Compute annualised Sharpe ratios
return_stats['sharpe_ratio_ANN'] = return_stats['sharpe_ratio_MON'] * np.sqrt(12)

In [ ]:
print(return_stats)

### Annualised Sharpe: Example of Fidelity Magellan Fund
https://fundresearch.fidelity.com/mutual-funds/performance-and-risk/316184100
 - Compare the Sharpe of Fidelity Magellan Fund to that of stocks/portfolios in our sample?
    - under what assumption(s)?
 - What is Fidelity Magellan's:
    - primary benchmark?
    - Morningstar category?
    - measures of total, systematic, and idiosyncratic risk?

### Summary statistics of factor returns

For each factor:
 - compute average realised monthly return and its standard deviation
 - cumulative monthly return

Focus on sample period between Jan 1970 and Dec 2024 (inclusive).

In [ ]:
# Compute mean and standard deviation of returns for each factor
factor_stats = factor_data[(factor_data['date'] >= "1970-01-01") & (factor_data['date'] <= "2024-12-31")].agg({
    'mktrf': ['mean', 'std'],
    'smb': ['mean', 'std'],
    'hml': ['mean', 'std'],
    'rmw': ['mean', 'std'],
    'cma': ['mean', 'std'],
    'mom': ['mean', 'std']
}).T.reset_index().rename(columns={'index': 'factor'})

print(factor_stats)

In [ ]:
# Compute correlations between factor returns
factor_returns_aux = factor_data[(factor_data['date'] >= "1970-01-01") & (factor_data['date'] <= "2024-12-31")][['mktrf', 'smb', 'hml', 'rmw', 'cma', 'mom']]
factor_correlation = factor_returns_aux.corr()
print(factor_correlation)

          mktrf       smb       hml       rmw       cma       mom
mktrf  1.000000  0.258882 -0.213797 -0.199254 -0.361695 -0.195169
smb    0.258882  1.000000  0.007218 -0.372369 -0.064578 -0.113241
hml   -0.213797  0.007218  1.000000  0.121142  0.691348 -0.185623
rmw   -0.199254 -0.372369  0.121142  1.000000  0.045230  0.076223
cma   -0.361695 -0.064578  0.691348  0.045230  1.000000  0.021304
mom   -0.195169 -0.113241 -0.185623  0.076223  0.021304  1.000000


In [ ]:
# Compute cumulative returns for each factor and risk-free rate using Dec 1969 as base month
# - subset relevant period
factor_data_cumul = factor_data[(factor_data['date'] >= "1969-12-01") & (factor_data['date'] <= "2024-12-31")].copy()

# - compute gross returns
factor_names = ['mktrf', 'smb', 'hml', 'rmw', 'cma', 'mom']
for name in factor_names:
    factor_data_cumul[name + '_gross'] = 1 + factor_data_cumul[name].fillna(0)
# - ensure that data is sorted by date
factor_data_cumul = factor_data_cumul.sort_values(by='date').reset_index(drop=True)
# - set the first observation to be the initial investment of 1 USD
for name in factor_names:
    factor_data_cumul.loc[factor_data_cumul['date'] == '1969-12-31', name + '_gross'] = 1
# - compute cumulative returns
for name in factor_names:
    factor_data_cumul[name + '_cumul'] = factor_data_cumul[name + '_gross'].cumprod()

In [ ]:
# Plot cumulative returns for each factor & risk-free rate
data_plot = factor_data_cumul[(factor_data_cumul['date'] >= "1969-12-01") & (factor_data_cumul['date'] <= "2024-12-31")].copy()
plt.figure(figsize=(12, 8))
for name in factor_names:
    plt.plot(data_plot['date'], data_plot[name + '_cumul'], label=name)
plt.title('Cumulative Returns of Fama-French Factors and Momentum Factor (Base: Dec 1969)')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid()

 - Momentum crashes:
Daniel, K., & Moskowitz, T. J. (2016). Momentum crashes. Journal of Financial Economics, 122(2), 221–247. https://doi.org/10.1016/j.jfineco.2015.12.002

### Sharpe ratios of factor returns

In [ ]:
# Compute Sharpe ratios of factor returns
factor_stats['sharpe_ratio_MON'] = factor_stats['mean'] / factor_stats['std']
factor_stats['sharpe_ratio_ANN'] = factor_stats['sharpe_ratio_MON'] * np.sqrt(12)
print(factor_stats)

### Estimation of Factor Models

#### Timeline of the development of modern multi-factor models:

 - (1973) Empirical tests of CAPM

 Fama, E. F., & MacBeth, J. D. (1973). Risk, Return, and Equilibrium: Empirical Tests. Journal of Political Economy, 81(3), 607–636. http://www.jstor.org/stable/1831028

 - (1993) Fama-French 3-factor model was proposed in:

Fama, E. F., & French, K. R. (1993). Common risk factors in the returns on stocks and bonds. Journal of Financial Economics, 33(1), 3–56. https://doi.org/10.1016/0304-405X(93)90023-5

 - (1993) Momentum factor was proposed by:

Jegadeesh, N., & Titman, S. (1993). Returns to buying winners and selling losers: Implications for stock market efficiency. The Journal of Finance, 48(1), 65–91. https://doi.org/10.1111/j.1540-6261.1993.tb04702.x

 - (1997) Carhart 4-factor model (Fama-French 3-factor + Momentum) was proposed in:

Carhart, M. M. (1997). On persistence in mutual fund performance. The Journal of Finance, 52(1), 57–82. https://doi.org/10.1111/j.1540-6261.1997.tb03808.x

 - (2015) Fama-French 5-factor model was proposed in:

Fama, E. F., & French, K. R. (2015). A five-factor asset pricing model. Journal of Financial Economics, 116(1), 1–22. https://doi.org/10.1016/j.jfineco.2014.10.010

 - (2023) Modern studies with <span style="color:red;">*parsimonious*</span> multi-factor models use so-called Fama-French 6-factor model which is FF5 + Momentum. See, for example:

Detzel, A., Novy-Marx, R., & Velikov, M. (2023). Model comparison with transaction costs. The Journal of Finance, 78(2), 665–708. https://doi.org/10.1111/jofi.13225

In [ ]:
# Import libraries needed for linear regression
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant

In [ ]:
# Print unique permnos with last comnam
unique_permnos = crsp_factors[['permno', 'comnam']].drop_duplicates()
print(unique_permnos)

#### CAPM

In [ ]:
# Run a linear regression via statsmodels
# - select one stock for the regression (since we are running time-series regression)
reg_data = crsp_factors[crsp_factors['permno'] == 11308]
# - drop rows with missing values
reg_data = reg_data.dropna(subset=['retex','mktrf'])

# Define dependent and independent variables
X = add_constant(reg_data['mktrf'])
y = reg_data['retex']
print(X.shape)
print(y.shape)

# Fit the OLS regression
# - compute Newey-West standard errors with automatic lag selection
model = OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags': None})
# - under i.i.d. errors:
# model = OLS(y, X).fit()

# Print the regression results summary
print(model.summary())

### Fama-French 6-factor model

In [ ]:
# Run a linear regression via statsmodels
# - select one stock for the regression (since we are running time-series regression)
reg_data = crsp_factors[crsp_factors['permno'] == 11308]
# - drop rows with missing values
reg_data = reg_data.dropna(subset=['retex','mktrf','smb','hml','rmw','cma','mom'])

# Define dependent and independent variables
X = add_constant(reg_data[['mktrf','smb','hml','rmw','cma','mom']])
y = reg_data['retex']
print(X.shape)
print(y.shape)

# Fit the OLS regression
# - compute Newey-West standard errors with automatic lag selection
model = OLS(y, X).fit(cov_type='HAC', cov_kwds={'maxlags': None})
# - under i.i.d. errors:
# model = OLS(y, X).fit()

# Print the regression results summary
print(model.summary())